In [22]:
import openai
import os
import pandas as pd

In [40]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [24]:
df = pd.read_csv("sales_data_sample.csv")

In [25]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [26]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [27]:
# Create a temporary database in RAM
temp_db = create_engine("sqlite:///:memory:", echo=True)

In [28]:
# Add df to temporary database as sql table
data = df.to_sql(name="sales",con=temp_db)

2023-03-07 10:33:19,917 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 10:33:19,918 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2023-03-07 10:33:19,919 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 10:33:19,920 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sales")
2023-03-07 10:33:19,920 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-07 10:33:19,921 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-07 10:33:19,923 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 10:33:19,924 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-03-07 10:33:19,924 INFO sqlalchemy

In [29]:
# Run an sql query with the connection to this sql database.
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT sum(SALES) FROM sales"))

2023-03-07 10:33:23,558 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 10:33:23,559 INFO sqlalchemy.engine.Engine SELECT sum(SALES) FROM sales
2023-03-07 10:33:23,559 INFO sqlalchemy.engine.Engine [generated in 0.00182s] ()
2023-03-07 10:33:23,561 INFO sqlalchemy.engine.Engine ROLLBACK


In [30]:
result.all()

[(10032628.85000001,)]

In [31]:
# Now, I need to create a prompt to deliver OpenAI the information about the table I have. I create a function for that.
def create_table(df):
    prompt = """### sqlite SQL table, with its properties:
    #
    # Sales({})
    #  
    """.format(",".join(str(col) for col in df.columns))
    return prompt

In [32]:
print(create_table(df))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #  
    


In [33]:
# For a non-technical person, create a prompt input
def prompt_input():
    npl_text = input("Enter the info you want:")
    return npl_text

In [34]:
prompt_input()

'Result of total sales per postalcode'

In [35]:
# Now, it is time to combine of them all
def combine_prompts(df, query):
    table_def = create_table(df)
    query_string = f"###A query to answer: {query}\nSELECT"
    return table_def+query_string

In [36]:
nlp_text = prompt_input()
prompt = combine_prompts(df,nlp_text)
# So in the end, this one will be send to GPT

# Muhtemelen bu Davinci ile text completion yapilarak denenecek. Sonra aynisini ChatGPT ile cagiracagim.

In [37]:
print(prompt)

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #  
    ###A query to answer: return the sum of sales per postalcode
SELECT


In [41]:
# Time to do the API call
response = openai.Completion.create(
    model = "code-davinci-002",
    prompt = combine_prompts(df, nlp_text),
    temperature = 0,
    max_tokens=150,
    top_p = 1.0,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = ['#',';']
)

In [45]:
response['choices'][0]['text']

' POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

In [46]:
# let's combine select with response now
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = "SELECT"+query
    return query

In [47]:
handle_response(response)

'SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

In [48]:
# Run the generated SQL code.
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-07 11:14:53,087 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-07 11:14:53,089 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE
2023-03-07 11:14:53,090 INFO sqlalchemy.engine.Engine [generated in 0.00258s] ()
2023-03-07 11:14:53,092 INFO sqlalchemy.engine.Engine ROLLBACK


In [49]:
result.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287

In [51]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-6rPM1kgJWrutQEnqME9qj1m2fOcHr at 0x1195a0410> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played at Globe Life Field in Arlington, Texas, which is the home ballpark of the Texas Rangers.",
        "role": "assistant"
      }
    }
  ],
  "created": 1678186865,
  "id": "chatcmpl-6rPM1kgJWrutQEnqME9qj1m2fOcHr",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 56,
    "total_tokens": 85
  }
}

In [52]:
# Let's try that with gpt-3.5 chat model API

openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages =[{"role":"user","content":"I have a database table named as Sales and it contains sale id, amount and postalcode as columns. Write a sql code that calcualtes the total sum of sales per postalcode."}],
    max_tokens=150
)

<OpenAIObject chat.completion id=chatcmpl-6rPNJmm3qnAWHXg6rMFhyPbweajx5 at 0x11942bd10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\nSELECT postalcode, SUM(amount) AS total_sales\nFROM Sales\nGROUP BY postalcode;",
        "role": "assistant"
      }
    }
  ],
  "created": 1678186945,
  "id": "chatcmpl-6rPNJmm3qnAWHXg6rMFhyPbweajx5",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 45,
    "total_tokens": 66
  }
}